In [1]:
import os
spark_home = os.path.abspath(os.getcwd() + "/spark/spark-3.5.5-bin-hadoop3")
hadoop_home = os.path.abspath(os.getcwd() + "/spark/winutils")
print(f"I am using the following SPARK_HOME: {spark_home}")
if os.name == 'nt':
    os.environ["HADOOP_HOME"] = f"{hadoop_home}"
    print(f"Windows detected: set HADOOP_HOME to: {os.environ['HADOOP_HOME']}")
    hadoop_bin = os.path.join(hadoop_home, "bin")
    os.environ["PATH"] = f"{hadoop_bin};{os.environ['PATH']}"
    print(f"  Also added Hadoop bin directory to PATH: {hadoop_bin}")

import findspark
import pyspark
from pyspark.streaming import StreamingContext

findspark.init(spark_home)
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession.builder.getOrCreate()


I am using the following SPARK_HOME: d:\OneDrive - CGIAR\Master\Advanced Analytics\assignments\assignment-03\spark\spark-3.5.5-bin-hadoop3
Windows detected: set HADOOP_HOME to: d:\OneDrive - CGIAR\Master\Advanced Analytics\assignments\assignment-03\spark\winutils
  Also added Hadoop bin directory to PATH: d:\OneDrive - CGIAR\Master\Advanced Analytics\assignments\assignment-03\spark\winutils\bin


In [3]:
raw_df = spark.read.json("data/raw")


In [4]:
raw_df.printSchema()


root
 |-- value: string (nullable = true)



In [5]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

# Define the schema for the nested JSON
paper_schema = StructType([
    StructField("aid", StringType()),
    StructField("title", StringType()),
    StructField("summary", StringType()),
    StructField("main_category", StringType()),
    StructField("categories", StringType()),
    StructField("published", StringType())
])


In [6]:
# Parse the nested JSON
papers_df = raw_df.withColumn("paper_data", 
                           from_json(col("value"), paper_schema))\
                 .select("paper_data.*")

# Display sample data
papers_df.show(5, truncate=False)


+---------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Count records
print(f"Total number of rows: {papers_df.count()}")


Total number of rows: 14594


In [12]:
dup_ids = papers_df.groupBy("aid").count().filter("count > 1")
print(f"Papers with duplicate IDs: {dup_ids.count()}")

if dup_ids.count() > 0:
    print("Sample of papers with duplicate IDs:")
    papers_df.join(dup_ids.select("aid"), "aid").orderBy("aid").show(5, truncate=False)


Papers with duplicate IDs: 4179
Sample of papers with duplicate IDs:
+---------------------------------+-------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
# Drop duplicates based on all columns instead of just "aid"
papers_unique_df = papers_df.dropDuplicates(["aid"])
print(f"After removing duplicates based on all columns: {papers_unique_df.count()} papers remain")


After removing duplicates based on all columns: 4442 papers remain


In [10]:
# See how many papers per category
papers_unique_df.groupBy("main_category").count().orderBy("count", ascending=False).show()


+-----------------+-----+
|    main_category|count|
+-----------------+-----+
|            cs.CV|  511|
|            cs.LG|  295|
|            cs.CL|  234|
|         quant-ph|  195|
|            cs.RO|  118|
|            cs.AI|  108|
|          math.AP|   94|
|          eess.SY|   91|
|cond-mat.mtrl-sci|   90|
|           hep-ph|   84|
|          math.OC|   73|
|            gr-qc|   70|
|   physics.optics|   69|
|          eess.SP|   68|
|          math.CO|   67|
|      astro-ph.GA|   66|
|            cs.CR|   62|
|           hep-th|   61|
|cond-mat.mes-hall|   60|
|          math.NA|   59|
+-----------------+-----+
only showing top 20 rows



In [11]:
papers_unique_df.write.mode("overwrite").json("data/interim")
